In [1]:
from fastMONAI.vision_all import *
from monai.networks.nets import UNet
from monai.apps import download_and_extract
from huggingface_hub import snapshot_download

In [2]:
# Create a custom loss function using Tversky Focal Loss
loss_func = CustomLoss(loss_func=TverskyFocalLoss(to_onehot_y=True, include_background=True, softmax=True))

# Create a UNet model with the specified parameters
model = UNet(spatial_dims=3, in_channels=1, out_channels=2, channels=(16, 32, 64, 128, 256), strides=(2, 2, 2, 2), num_res_units=2).model

In [3]:
DATA_DIR = Path('./data')
STUDY_DIR = download_example_spine_data(path=DATA_DIR)

Downloading...
From: https://drive.google.com/uc?id=1K1J2wmWxYUPY0NbylcV4Otv88EqCL4Ys
To: /var/folders/t6/1sf85j5s5pbgsfpgxrk8t3m00000gn/T/tmpjvf560bp/spine_example_data.zip
100%|██████████| 2.52M/2.52M [00:00<00:00, 48.4MB/s]

2024-06-05 15:11:49,628 - INFO - Downloaded: spine_example_data.zip
2024-06-05 15:11:49,628 - INFO - Expected md5 is None, skip md5 check for file spine_example_data.zip.
2024-06-05 15:11:49,629 - INFO - Non-empty folder exists in data/spine_example_data, skipped extracting.


In [25]:
# Create a dictionary containing the example data
example_data = {'t2_img_path':str(STUDY_DIR/'img.nii.gz'), 't2_mask_path':str(STUDY_DIR/'pred_mask.nii.gz'), 'subject_id':-1, 'is_val':False}

# Create a DataFrame from the example data dictionary (remove [] if data > 1) 
df = pd.DataFrame([example_data])
df

,t2_img_path,t2_mask_path,subject_id,is_val
0,data/spine_example_data/img.nii.gz,data/spine_example_data/pred_mask.nii.gz,-1,False


In [5]:
# Set the data directory to the specified path relative to the working directory
#TRAIN_DATA_DIR = DATA_DIR/'ADD_PATH'

In [6]:
# Set the fold number
#fold = 0

# Read the data from the specified CSV file
#df = pd.read_csv(TRAIN_DATA_DIR/'ADD_CSV_FILENAME')

# Set the 'is_val' column to True for rows in the specified fold
#df.loc[df.fold == fold, 'is_val'] = True

In [26]:
med_dataset = MedDataset(img_list=df.t2_img_path.tolist(), reorder=True, max_workers=12)
med_dataset

In [27]:
data_info_df = med_dataset.summary()
data_info_df

,dim_0,dim_1,dim_2,voxel_0,voxel_1,voxel_2,orientation,example_path,total
0,16,400,400,4.4,0.78,0.78,RAS+,data/spine_example_data/img.nii.gz,1


In [28]:
# Get the suggested resampling and reordering values from the dataset
resample, reorder = med_dataset.suggestion()
resample, reorder

([4.4, 0.78, 0.78], True)

In [29]:
# Get the largest image size from the dataset, using the specified resampling values
img_size = med_dataset.get_largest_img_size(resample=resample)
img_size

[16.0, 400.0, 400.0]

In [30]:
size = [16,400,400]
bs = 4

In [31]:
# Create a list of transformations to apply to the images, with a probability of 0.5 for each transformation being applied
item_tfms = [ZNormalization(), PadOrCrop(size), 
             RandomGamma(log_gamma=(-0.2,0.2), p=0.5), RandomAffine(scales=(0.9, 1.1), degrees=5, p=0.5), 
             RandomElasticDeformation(num_control_points=5, max_displacement=5.5, p=0.5)]

In [32]:
# Create a MedDataBlock with the specified blocks, splitter, and item transformations
dblock = MedDataBlock(blocks=(ImageBlock(cls=MedImage), MedMaskBlock), 
                      splitter=ColSplitter('is_val'),
                      get_x=ColReader('t2_img_path'),
                      get_y=ColReader('t2_mask_path'),
                      item_tfms=item_tfms,
                      reorder=reorder,
                      resample=resample) 

In [35]:
dls = dblock.dataloaders(df, bs=bs)

In [36]:
# training and validation
len(dls.train_ds.items), len(dls.valid_ds.items)

(1, 0)

In [37]:
dls.show_batch(unique=True, max_n=1, figsize=(5, 5)) 

ValueError: This DataLoader does not contain any batches

In [38]:
learn = Learner(dls, model, loss_func=loss_func, opt_func=ranger, metrics=[multi_dice_score])

In [39]:
lr=learn.lr_find()
lr 

ZeroDivisionError: integer division or modulo by zero